# GTFS

## GTFS VINTRA

In [121]:
import pandas as pd
from itables import init_notebook_mode
import os
import subprocess

init_notebook_mode(all_interactive=True)


<IPython.core.display.Javascript object>

In [122]:
working_directory = f'{os.getcwd()}/../../data/gtfs/vintra/'
gtfs_files_directory = f'{working_directory}/gtfs-files/'

gtfs_file_stats_df = pd.DataFrame()

for file in sorted(os.listdir(gtfs_files_directory)):
    if file.endswith('.zip'):
        filename, _, _ = file.partition('.zip')

        p = subprocess.Popen([
            f'java -jar gtfs-validator-301.jar -i gtfs-files/{file} -o reports -v {filename}_report.json -e {filename}_system_errors.json -n -c lt'],
            cwd=working_directory, shell=True, stdout=subprocess.PIPE,
            stderr=subprocess.PIPE)
        out, err = p.communicate()
        errcode = p.returncode

        _, _, gtfs_files_txt = out.decode("utf-8").partition('seconds\n')
        gtfs_files = gtfs_files_txt.splitlines()

        gtfs_files_dict = {'failas': filename}
        for gtfs_file_rep in gtfs_files:
            gtfs_file, c = gtfs_file_rep.split('\t')
            gtfs_files_dict[gtfs_file] = c if c != 'MISSING_FILE' else None

        gtfs_file_stats_df = gtfs_file_stats_df.append(gtfs_files_dict, ignore_index=True, )

gtfs_file_stats_df = gtfs_file_stats_df.reindex(
    columns=[
        'failas',
        'agency.txt',
        'calendar.txt',
        'calendar_dates.txt',
        'routes.txt',
        'shapes.txt',
        'stop_times.txt',
        'stops.txt',
        'trips.txt',
        'fare_attributes.txt',
        'fare_rules.txt',
        'attributions.txt',
        'feed_info.txt',
        'frequencies.txt',
        'levels.txt',
        'pathways.txt',
        'transfers.txt',
        'translations.txt'
    ]
).set_index('failas')


gtfs_file_stats_df[
    [
        'agency.txt',
        'calendar.txt',
        'calendar_dates.txt',
        'routes.txt',
        'shapes.txt',
        'stop_times.txt',
        'stops.txt',
        'trips.txt',
        'fare_attributes.txt',
        'fare_rules.txt',
    ]
] = gtfs_file_stats_df[
    [
        'agency.txt',
        'calendar.txt',
        'calendar_dates.txt',
        'routes.txt',
        'shapes.txt',
        'stop_times.txt',
        'stops.txt',
        'trips.txt',
        'fare_attributes.txt',
        'fare_rules.txt',
    ]
].fillna('❌')

gtfs_file_stats_df.fillna('⚠️', inplace=True)
gtfs_file_stats_df.style.set_sticky(axis="index")

gtfs_file_stats_df

Loading... (need help?)


In [125]:
import json

reports_dir = f'{working_directory}/reports/'

gtfs_notices_df = pd.DataFrame()


for file in sorted(os.listdir(reports_dir)):
    if file.endswith('report.json'):
        gtfs_filename, _, _ = file.partition('_report.json')

        with open(os.path.join(reports_dir, file)) as fp:
                data = json.load(fp)

                for notice in data['notices']:
                    gtfs_notices_df = gtfs_notices_df.append({
                        'failas': gtfs_filename,
                        'klaida': notice['code'],
                        'sunkumas': notice['severity'],
                        'viso': notice['totalNotices'],
                    }, ignore_index=True, )


gtfs_notices_df

Loading... (need help?)


In [124]:
# import gtfs_functions as gtfs
# import keplergl as kp
#
# routes, stops, stop_times, trips, shapes = gtfs.import_gtfs("../../data/gtfs/vintra/PanevezioM.zip", busiest_date=True)
#
# cutoffs = [0, 6, 9, 15.5, 19, 22, 24]
# line_freq = gtfs.lines_freq(stop_times, trips, shapes, routes, cutoffs=cutoffs)
# line_freq.head()
